# Coalitional Games: Representation, Solution concepts, and Issues
In this notebook, we will explore how to use computation to explore and solve coalitional games.
Coalitional games, also known as Transferable Utility (TU) games, are a particular class of games where the payoff is not given to a single player but to a group (coalition) of players.

Utility is *transferable* if one player can transfer part of their utility to another player, without loss. Such transfers are possible if the players have a common currency that is valued equally by all the players. It's important to note that we are still talking about utility. So, for instance, being able to transfer cash payoffs does not imply that *utility* is transferable: wealthy and poor players may derive a different utility from the same amount of money.

When a group of players joins forces to reach a common goal, for example by pooling their money to buy a common good, the players need to decide how to divide the payoff. We will see how to answer this question! </br>
We might also want to know what coalitions are likely to emerge. This might be a tough question, since if we have n players, then we have $2^n$ possible coalitions.

## An example of coalitional game: Buying ice-cream
Let's consider a coalitional game where we have three players (Alice, Bob, and Charlie) who want to buy and divide ice-cream.</br>
Each player has a fixed amount of money:
* A = $6
* B = $3
* C = $3

Three types of ice-cream tubs are for sale:
* Type 1 costs $7, contains 500g
* Type 2 costs $9, contains 750g
* Type 3 costs $11, contains 1000g

The payoff of each group is the maximum quantity of ice-cream the members of the group can buy by pooling their money </br>
The ice-cream can be shared arbitrarily within the group.

### Possible coalitions and their payoffs
No single person, with the money available, can buy any amount of ice-cream.</br>
- $v(\emptyset) = v(\{ \textrm{A} \}) = v(\{\textrm{B}\}) = v(\{\textrm{C}\}) = 0$

No coalition without Alice can buy any ice-cream
- $v(\{ \textrm{A, B} \}) = 750\textrm{g}, v(\{ \textrm{A, C} \}) = 750\textrm{g}, v(\{ \textrm{B, C} \}) = 0$
- $v(\{ \textrm{A, B, C} \}) = 1000\textrm{g}$

What we have just written here is the so called *characteristic function*.

## Formalisation of coalitional games
> **Definition (Coalitional Game):** </br>
>A coalitional game (transferable utility) game is a pair $G = (N, v)$, where:
>- $N =\{1, ..., n\}$ is the set of players
>- $v$: $2^N \longrightarrow \mathbb{R}$ is the characteristic function
>- for each subset of players $C$, $v(C)$ is the amount that the members of $C$ can earn by working together

We usually assume that $v$ is
- normalized: $v(Ø)$ = 0
- non-negative: $v(C) \ge 0, \quad \forall \, C \subseteq N$
- monotone: $v(C) \le v(D), \quad \forall \; C, D \quad \mathrm{such\;that} \quad C \subseteq D$

A coalition is any subset of N; N itself is called the grand coalition

### Representing a characteristic function in Python
To represent a characteristic function in Python, in a naive way, we can use a dictionary where we associate a value to each coalition.
We can use frozen sets. Frozen sets are unchangeable sets, so they can be hashed. The convenient thing about frozen sets is that they offer us all the useful and fast functions offered by sets, such as checking that an element is a member of a set or checking if a set $S$ is the subset of another set $T$, and they are also hashable so they can be used as keys in a dictionary.

<!---
(A) = 0 </br>
%(B) = 0 </br>
(C) = 0 </br>
(A,B) = 750 </br>
(A,C) = 750 </br>
(B,C) = 0 </br>
(A,B,C) = 1000 </br>
-->

In [1]:
# This is the characteristic function for the ice-cream game we have just described, using frozen sets
v = {
    frozenset(['A']): 0,
    frozenset(['B']): 0,
    frozenset(['C']): 0,
    frozenset(['A', 'B']): 750,
    frozenset(['A', 'C']): 750,
    frozenset(['B', 'C']): 0,
    frozenset(['A', 'B', 'C']): 1000
}

In [2]:
v

{frozenset({'A'}): 0,
 frozenset({'B'}): 0,
 frozenset({'C'}): 0,
 frozenset({'A', 'B'}): 750,
 frozenset({'A', 'C'}): 750,
 frozenset({'B', 'C'}): 0,
 frozenset({'A', 'B', 'C'}): 1000}

### Superadditive games
> **Definition (Superadditive games):** </br>
A characteristic function game $G(N,v)$ is said to be superadditive if it satisfies $v(C_1 \cup C_2) \ge v(C_1) + v(C_2)\; \forall C_1, C_2 \subset N\; \text{such that}\; C_1 \cap C_2 = \emptyset$.

Let's implement a function that recognises if a game is superadditive or not. First, we need a function to generate all the possible coalitions of a set on $N$ players. That is, we need a function to generate the powerset of a set (or list) of elements. Then, following the definition, for each couple of coalitions $C_1$ and $C_2$ in this powerset, if the intersection between these coalitions is the empty set, we check that the condition $v(C_1 \cup C_2) \ge v(C_1) + v(C_2)$ holds. If it doesn't, the game is not superadditive.

In [3]:
from itertools import combinations

def intersection(c1,c2):
    intersection_elements = []
    for element in c1:
        if element in c2:
          intersection_elements.append(element)
    return frozenset(intersection_elements)

def is_superadditive(characteristic_function):
    condition = True

    for c1 in characteristic_function:
      for c2 in characteristic_function:
        if c1 != c2 and len(intersection(c1,c2)) == 0:
          if characteristic_function[c1 | c2] < characteristic_function[c1] + characteristic_function[c2]:
            condition = False
    return condition

In [4]:
v = {
    frozenset(['A']): 0,
    frozenset(['B']): 0,
    frozenset(['C']): 0,
    frozenset(['A', 'B']): 750,
    frozenset(['A', 'C']): 750,
    frozenset(['B', 'C']): 0,
    frozenset(['A', 'B', 'C']): 1000
}


# This game is superadditive, so this should return True
print(is_superadditive(v))

True


In [5]:
v = {
    frozenset(['A']): 6,
    frozenset(['B']): 12,
    frozenset(['C']): 42,
    frozenset(['A', 'B']): 12,
    frozenset(['A', 'C']): 42,
    frozenset(['B', 'C']): 42,
    frozenset(['A', 'B', 'C']): 42
}

# This game is not superadditive, so this should return False
print(is_superadditive(v))

False


## Solution concepts: the core
>**Definition (Core):** </br>
>The core of a game is the set of all stable outcomes; that is, outcomes that no coalition wants to deviate from. </br>
>More formally, the core is defined as follows:
>
>$$
    \mathrm{core}(G) =  \left\{\quad \mathbf{x}  \quad \left| \;
        \begin{align}
            &x_i \ge 0\, \forall i \in N &\text{(The payoff is positive for each player)} \nonumber\\
            &\sum_{i \in N} x_i = v(N) \qquad &\text{(Efficiency: all value is allocated)} \nonumber\\
            &\sum_{i \in C} x_i \ge v(C)\, \forall C \subseteq N \qquad &\text{(Stability: No coalition wants to deviate)} \nonumber \\
        \end{align}
    \right\}\right.
>$$

Computing the core is a difficult task. Enumerating all the possible outcomes in the core is clearly unfeasible, even if we consider only integer payoffs. A possibility is to find the extreme points that define the region of the core, but that requires to solve a sequence of linear programs (LP). This is clearly beyond the goals of this course. Additionally, since the core has important limitations and it can also be empty for some games, such an effort would not be worth.</br>
We will, instead, use the aforementioned definition of core to check whether an outcome is in the core. This will also help us to understand some limitations of stable outcomes.

In [8]:
from itertools import combinations

# A function to generate the powerset: it returns a list of 2^(n-1) frozen sets
def powerset(List):
    subs = [frozenset(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return subs

# This function checks if an outcome for a given game is stable
def is_stable(outcome, characteristic_function):
    for c in characteristic_function:
      value = characteristic_function[c]
      allocated = 0
      for player in c:
        allocated += outcome[player]
      if allocated < value:
        return False
    return True


# This function applies the definition of core to check whether an outcome is in the core of a game
def is_in_the_core(outcome, characteristic_function):
    positivity = True
    efficient = True

    # check for positivity
    for x in outcome:
        if outcome[x] < 0:
            is_positive = False
            return False

    # check for efficiency
    if int(sum(outcome.values())) != characteristic_function[max(characteristic_function)]:
      return False

    # check for stability
    if not is_stable(outcome, characteristic_function):
      return False

    return True



In [13]:
# Now, to define our coalitional game we need a characteristic function.
v = {
    frozenset(['A']): 0,
    frozenset(['B']): 0,
    frozenset(['C']): 0,
    frozenset(['A', 'B']): 750,
    frozenset(['A', 'C']): 750,
    frozenset(['B', 'C']): 0,
    frozenset(['A', 'B', 'C']): 1000
}

# Now, we need to define an outcome in order to test if it is in the core.
# We start with an equal redistribution
x = {
    'A': 333.3334,
    'B': 333.3333,
    'C': 333.3333
}

# It should return False, since v({A,B}) > x(A) + x(B)
print(is_in_the_core(x, v))

False


In [11]:
# We try with another outcome: since A has more money than B and C, we give a higher payoff to A
x = {
    'A': 500,
    'B': 250,
    'C': 250
}

# It should return True, since v({B,C}) = 0 and x(B) + x(C) = 500
print(is_in_the_core(x, v))

True


In [12]:
# We try with a very unfair outcome (B and C pay for A to take all!)
x = {
    'A': 1000,
    'B': 0,
    'C': 0
}

# It should return True, since v(B) = v(C) = v({B,C}) = 0 which is equal to x(B) + x(C)
# This shows one limitation of the core. Such an unfair outcome is in the core!
print(is_in_the_core(x, v))

True


## Solution concepts: the Shapley Value
>**Definition (Shapley Value):** </br>
>The Shapley value is a way to redistribute the payoff obtained by the coalition among its members. The idea behind the Shapley value is to redistribute the payoff proportionally to the contribution of each player to the coalition. A possible solution consists in assigning to each player the average of all their marginal contributions over the possible orderings of the players in the coallition. That is:
>
>$$
\phi(i, v) = \frac{1}{|N|!} \sum_{\pi \in \Pi_N} v(\mathrm{B}(\pi, i) \cup \{i\}) - v(\mathrm{B}(\pi, i))
>$$
>
>Where: </br>
$\phi(i, v)$ is the Shapley value (i.e. the payoff) for the player $i$ </br>
$v$ is the characteristic function </br>
$\Pi_N$ is the set of all possible orderings of the elements in $N$; that is, the set of all the possible permutations of $N$ </br>
$\mathrm{B}(\pi, i)$ is the set of predecessors of $i$ in the permutation under consideration.

There are other two, equivalent, equations to compute the Shapley value:

$$
\phi(i, v) = \sum_{S \subseteq N} \frac{(|N| - |S|)! \times (|S| - 1)!}{|N|!} (v(S) - v(S \setminus \{i\}))
$$
$$
\phi(i, v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|! \times (|N| - |S| - 1)!}{|N|!} (v(S \cup \{i\}) - v(S))
$$

Note that in the last equation, the summation is over all the subsets $S \subseteq N$ which do not contain the player $i$.

The Shapley value satisfies the following properties:
- Players receive, at least, the same payoff they would receive if they did not participate to the coalition: $\phi(i, v) \ge v(\{i\})\; \forall i \in N$
- The Shapley value is Pareto efficient: $\sum_{i \in N} \phi(i, v) = v(N)$
- Players with the same marginal contributions receive the same payoff: $v(S \cup \{i\}) = v(S \cup \{j\})\; \forall S \in N \setminus \{i,j\} \Longrightarrow \phi(i, v) = \phi(j, v)$
- Players with marginal contribution equal to zero (such players are known as *null players*) receive payoff equal to zero: $v(S) = v(S \cup \{i\})\; \forall S \in N \setminus \{i\} \Longrightarrow \phi(i,v) = 0$
- The Shapley value is additive. Let $G_1 = (N, v)$ and $G_2 = (N, w)$ be two games defined on the same set of players: $\phi(i, v + w) = \phi(i, v) + \phi(i, w)\; \forall i \in N$

It can be shown that the Shapley value is the only payoff distribution method which satisfies the aforementioned properties.

The Shapley value, however, has one important limitation: it is not necessarily a stable solution. As such, it is not guaranteed that $\sum_{i \in C} \phi(i, v) \ge v(C)\, \forall C \subseteq N$.

### Computing the Shapley value for the Ice-cream game

Let's compute the Shapley value for this characteristic function: </br>
- $v(\emptyset) = v(\{ \textrm{A} \}) = v(\{\textrm{B}\}) = v(\{\textrm{C}\}) = 0$
- $v(\{ \textrm{A, B} \}) = 750\textrm{g}, v(\{ \textrm{A, C} \}) = 750\textrm{g}, v(\{ \textrm{B, C} \}) = 0$
- $v(\{ \textrm{A, B, C} \}) = 1000\textrm{g}$

We just need to compute the average marginal contribution of each player. Let's denote the marginal contribution of a player $i \in N$, for a given permutation $\pi \subseteq \Pi_N$ as $\delta_i(\pi)$

<b>Computation for player A</b>

$
\begin{aligned}
&\pi = \text{(A, B, C):} \qquad &\delta_A(\pi) &= v(\{A\}) - v(\{\emptyset\}) &=& &0& \\
&\pi = \text{(A, C, B):} &\delta_A(\pi) &= v(\{A\}) - v(\{\emptyset\}) &=& &0& \\
&\pi = \text{(B, A, C):} &\delta_A(\pi) &= v(\{A, B\}) - v(\{B\}) = 750 - 0 &=& &750& \\
&\pi = \text{(B, C, A):} &\delta_A(\pi) &= v(\{A, B, C\}) - v(\{B, C\}) = 1000 - 0 &=& &1000& \\
&\pi = \text{(C, A, B):} &\delta_A(\pi) &= v(\{A, C\}) - v(\{C\}) = 750 - 0 &=& &750& \\
&\pi = \text{(C, B, A):} &\delta_A(\pi) &= v(\{A, B, C\}) - v(\{B, C\}) = 1000 - 0 &=& &1000&
\end{aligned}
$

$$\phi(A, v) = (750 + 1000 + 750 + 1000)/6 = 3500/6 = 583.\overline{33}$$

<b>Computation for player B (and C)</b>

$
\begin{aligned}
&\pi = \text{(A, B, C):} \qquad &\delta_B(\pi) &= v(\{A, B\}) - v(\{A\}) = 750 - 0 &=& &750& \\
&\pi = \text{(A, C, B):} &\delta_B(\pi) &= v(\{A, B, C\}) - v(\{A, C\}) = 1000 - 750 &=& &250& \\
&\pi = \text{(B, A, C):} &\delta_B(\pi) &= v(\{B\}) - v(\{\emptyset\}) &=& &0& \\
&\pi = \text{(B, C, A):} &\delta_B(\pi) &= v(\{B\}) - v(\{\emptyset\}) &=& &0& \\
&\pi = \text{(C, A, B):} &\delta_B(\pi) &= v(\{A, B, C\}) - v(\{A, C\}) = 1000 - 750 &=& &250& \\
&\pi = \text{(C, B, A):} &\delta_B(\pi) &= v(\{B, C\}) - v(\{C\}) &=& &0&
\end{aligned}
$

$$\phi(B, v) = \phi(C, v) = (750 + 250 + 250)/6 = 1250/6 = 208.\overline{33}$$

As such, $\phi = (583.\overline{33}, 208.\overline{33}, 208.\overline{33})$

### Implementing the Shapley value in Python
We will now implement the Shapley value computation. We could use any of the aforementioned formula, but the second formula is better for the exact Shapley value computation. In fact, for $n$ players we have a total of $n!$ permutations versus $2^n$ coalitions. As such, implementing the computation of the Shapley values with the second formula is faster than implementing it with the first formula, because the second formula leads to an algorithm with time complexity $\mathcal{O}(2^n)$ while the first formula leads to an $\mathcal{O}(n!)$ algorithm. We will start by generating the powerset of players (that is, all the possible coalitions) and we are going to compute the marginal contributions.

In [49]:
from math import factorial
# A function to generate the powerset: it returns a list of 2^(n-1) frozen sets
def powerset(List):
    subs = [frozenset(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return subs

# We now implement the Shapley value for a given player, using the second equation
def shapley_value(player, characteristic_function):
    coalitions_without_player = {c for c in characteristic_function if player not in c}
    n = max(characteristic_function)

    total = 0

    for s in coalitions_without_player:
      part1 = (factorial(len(s))*factorial(len(n)-len(s)-1))/factorial(len(n))
      part2 = characteristic_function[s.union(frozenset([player]))] - characteristic_function[s]
      total += part1*part2

    return total

# This function returns a dictionary with the Shapley value for each player
def shapley(characteristic_function):
    # To get the grand coalition from the characteristic function, we can use the function max
    return {player: shapley_value(player, characteristic_function) for player in max(characteristic_function)}

In [48]:
v = {
    frozenset(['A']): 0,
    frozenset(['B']): 0,
    frozenset(['C']): 0,
    frozenset(['A', 'B']): 750,
    frozenset(['A', 'C']): 750,
    frozenset(['B', 'C']): 0,
    frozenset(['A', 'B', 'C']): 1000
}

# As we have seen before, it should be (583.33, 208.33, 208.33)
shapley(v)

{'B': 208.33333333333331, 'C': 208.33333333333331, 'A': 583.3333333333333}

## Computational issues of Coalitional Game Theory and naive representations of the characteristic function
We face two computational problems in Coalitional Game Theory:
1. The naive representation of the characteristic function is exponential in the number of players n: We need to list all the $2^n$ coalitions;
2. Computationally expensive algorithms: Checking the stability of an outcome or computing the Shapley value require to go over $2^n$ coalitions.

In general, we have the following strategies to tackle these two problems:
1. Focusing on *restricted classes* of games; for instance, games on combinatorial structures such as routing games. The problem here is that we give up on representing other interesting games.
2. Developing approximated algorithms to solve games; for instance, we can approximate the computation of the Shapley value. We give up on the exact solution but we reduce the computational time.
3. Using compact representations for games; instead of representing the characteristic function in a naive way, we can use networked structures to obtain compact but expressive representations.

We will see some of these strategies in the next lesson!